# 决策树

In [5]:
import numpy as np
from collections import Counter

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        # 特征索引
        self.feature = feature
        # 特征阈值
        self.threshold = threshold
        # 左子树
        self.left = left
        # 右子树
        self.right = right
        # 叶节点值
        self.value = value

class DecisionTree:
    def __init__(self, max_depth=10, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root = None
        
    def fit(self, X, y):
        """训练决策树"""
        self.n_classes = len(np.unique(y))
        self.root = self._grow_tree(X, y)
        
    def _grow_tree(self, X, y, depth=0):
        """递归构建决策树"""
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))
        
        # 检查停止条件
        if (self.max_depth is not None and depth >= self.max_depth) or \
           n_samples < self.min_samples_split or \
           n_labels == 1:
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)
        
        # 寻找最佳分割点
        feature_idxs = np.arange(n_features)
        best_feature, best_threshold = self._best_split(X, y, feature_idxs)
        
        # 创建子节点
        left_idxs = X[:, best_feature] <= best_threshold
        right_idxs = X[:, best_feature] > best_threshold
        left = self._grow_tree(X[left_idxs], y[left_idxs], depth + 1)
        right = self._grow_tree(X[right_idxs], y[right_idxs], depth + 1)
        
        return Node(best_feature, best_threshold, left, right)
    
    def _best_split(self, X, y, feature_idxs):
        """寻找最佳分割特征和阈值"""
        best_gain = -1
        split_feature, split_threshold = None, None
        
        for feature_idx in feature_idxs:
            X_column = X[:, feature_idx]
            thresholds = np.unique(X_column)
            
            for threshold in thresholds:
                gain = self._information_gain(y, X_column, threshold)
                
                if gain > best_gain:
                    best_gain = gain
                    split_feature = feature_idx
                    split_threshold = threshold
                    
        return split_feature, split_threshold
    
    def _information_gain(self, y, X_column, threshold):
        """计算信息增益"""
        parent_entropy = self._entropy(y)
        
        # 生成左右子树
        left_idxs = X_column <= threshold
        right_idxs = X_column > threshold
        
        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0
        
        # 计算加权平均熵
        n = len(y)
        n_l, n_r = len(y[left_idxs]), len(y[right_idxs])
        e_l, e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
        child_entropy = (n_l/n) * e_l + (n_r/n) * e_r
        
        return parent_entropy - child_entropy
    
    def _entropy(self, y):
        """计算熵"""
        hist = np.bincount(y)
        ps = hist / len(y)
        return -np.sum([p * np.log(p) for p in ps if p > 0])
    
    def _most_common_label(self, y):
        """返回最常见的标签"""
        counter = Counter(y)
        return counter.most_common(1)[0][0]
    
    def predict(self, X):
        """预测样本的类别"""
        return np.array([self._traverse_tree(x, self.root) for x in X])
    
    def _traverse_tree(self, x, node):
        """遍历决策树进行预测"""
        if node.value is not None:
            return node.value
        
        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)

# 生成示例数据
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# 生成分类数据
X, y = make_classification(
    n_samples=1000, 
    n_features=10,
    n_classes=2,
    random_state=42
)

print(X.shape, y.shape)

# 分割训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 创建和训练决策树
tree = DecisionTree(max_depth=5)
tree.fit(X_train, y_train)

# 预测
predictions = tree.predict(X_test)

# 计算准确率
accuracy = np.mean(predictions == y_test)
print(f"Accuracy: {accuracy:.4f}")



(1000, 10) (1000,)
Accuracy: 0.8750


In [9]:
import xgboost as xgb
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

params ={
    'max_depth':5,
    'eta':0.3,
    'objective': 'binary:logistic',
    'eval_metric': 'logliss'
}

num_rounds =100

# 设置早停机制
early_stopping_rounds = 10
evals = [(dtrain, 'train'), (dtest, 'eval')]

model = xgb.train(
    params,
    dtrain,
    num_rounds,
    evals=evals,
    early_stopping_rounds=early_stopping_rounds,
    verbose_eval=10  # 每10轮打印一次评估结果
)

predictions = model.predict(dtest)

pre_label = [1 if p>0.5 else 0 for p in predictions]
accuracy = accuracy_score(y_test, pred_labels)
print(f"\nAccuracy: {accuracy:.4f}")


XGBoostError: [00:39:14] /Users/runner/work/xgboost/xgboost/src/metric/metric.cc:49: Unknown metric function logliss
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x0000000147144428 dmlc::LogMessageFatal::~LogMessageFatal() + 124
  [bt] (1) 2   libxgboost.dylib                    0x00000001473695e8 xgboost::Metric::Create(std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char>> const&, xgboost::Context const*) + 124
  [bt] (2) 3   libxgboost.dylib                    0x000000014733d764 xgboost::LearnerConfiguration::ConfigureMetrics(std::__1::vector<std::__1::pair<std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char>>, std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char>>>, std::__1::allocator<std::__1::pair<std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char>>, std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char>>>>> co